In [2]:
!pip install langchain
!pip install langchain-experimental
!pip install psycopg2-binary 
!pip install openai
!pip install google-api-python-client
!pip install google-search-results

In [2]:
import os
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain import OpenAI, SQLDatabase
from langchain.chat_models import ChatOpenAI
from langchain_experimental.sql import SQLDatabaseChain
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain.memory.chat_message_histories import RedisChatMessageHistory
from langchain import OpenAI, LLMChain
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.prompts.prompt import PromptTemplate
from typing import Any, Dict, List
from langchain.schema import BaseMemory
from langchain_experimental.generative_agents.memory import GenerativeAgentMemory
from langchain.memory.buffer import ConversationBufferMemory
from langchain.chains.conversation.base import ConversationChain

import warnings
warnings.filterwarnings("ignore")

In [3]:
OPENAI_API_KEY=""
SERPAPI_API_KEY = ""
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["SERPAPI_API_KEY"] = SERPAPI_API_KEY

# #Task 1 - Converting Text to SQL Queries using LLM on Langchain

In [4]:
DB_USER=""
DB_PASSWORD=""
DB_HOST=""
DB_PORT=""
DB_NAME=""

db = SQLDatabase.from_uri(
    f'postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}',
)

# setup llm
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

# Create db chain
prompt_query = """
            Given an input question, first create a syntactically correct postgresql query to run, then look at the results of the query and return the answer.

            
                
            Here is the format to follow:
            
            Question: Question here
            
            SQLQuery: SQL Query to run
            
            SQLResult: Result of the SQLQuery
            
            Answer: Final answer here
            
            {question}
      """
# Setup the database chain
db_chain = SQLDatabaseChain(llm=llm,
                            database=db,
                            verbose=True,
                            use_query_checker=True)


In [5]:
question = "how much did TSLA's closing stock price on October 19, 2023?"
query = prompt_query.format(question=question)
db_chain.run(query)



> Entering new SQLDatabaseChain chain...

            Given an input question, first create a syntactically correct postgresql query to run, then look at the results of the query and return the answer.

            
                
            Here is the format to follow:
            
            Question: Question here
            
            SQLQuery: SQL Query to run
            
            SQLResult: Result of the SQLQuery
            
            Answer: Final answer here
            
            how much did TSLA's closing stock price on October 19, 2023?
      
SQLQuery:SELECT close_price
FROM llm_stock
WHERE symbol = 'TSLA' AND date = '2023-10-19';
SQLResult: [(220.11,)]
Answer:The closing stock price of TSLA on October 19, 2023 was 220.11.
> Finished chain.


'The closing stock price of TSLA on October 19, 2023 was 220.11.'

# #Task 2- Google Search Queries

In [6]:
llm = OpenAI(temperature=0, model_name='gpt-4')
tools = load_tools(["serpapi"], llm=llm)

agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [7]:
question = "How much the Tesla stock price recently?"
agent.run(input=question)



> Entering new AgentExecutor chain...
I need to look up the current price of Tesla stock.
Action: Search
Action Input: "current Tesla stock price"
Observation: {'type': 'finance_results', 'title': 'Tesla Inc', 'exchange': 'NASDAQ', 'stock': ' TSLA', 'currency': 'USD', 'price': 197.36, 'previous_close': 207.3}
Thought:I now know the final answer.
Final Answer: The recent price of Tesla stock is $197.36.

> Finished chain.


'The recent price of Tesla stock is $197.36.'

In [8]:
llm("How much the Tesla stock price recently?")

"As an AI, I don't have real-time data. Please check the latest financial news or stock market apps for the most recent price of Tesla stock."

# #Task 3 History Chatting

In [12]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain.memory import ConversationBufferMemory, ReadOnlySharedMemory
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.utilities import SerpAPIWrapper

In [28]:
template = """This is a conversation between a human and a bot:

{chat_history}

Write a summary of the conversation for {input}:
"""

prompt = PromptTemplate(input_variables=["input", "chat_history"], template=template)
memory = ConversationBufferMemory(memory_key="chat_history")
readonlymemory = ReadOnlySharedMemory(memory=memory)
summary_chain = LLMChain(
    llm=OpenAI(),
    prompt=prompt,
    verbose=True,
    memory=readonlymemory,
)
search = SerpAPIWrapper()

tools = [
    Tool(
        name="Summary",
        func=summary_chain.run,
        description="useful for when you summarize a conversation. The input to this tool should be a string, representing who will read this summary.",
    ),
]


In [29]:
prefix = """Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:"""
suffix = """Begin!"

{chat_history}
Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "chat_history", "agent_scratchpad"],
)

llm_chain = LLMChain(llm=ChatOpenAI(temperature=0, model='gpt-4'), prompt=prompt)
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)
agent_chain = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True, memory=memory
)

In [30]:
agent_chain.run("Hi my name is jihwan")



> Entering new AgentExecutor chain...
Thought: The user is introducing themselves. No specific question has been asked, but a polite response is appropriate.
Action: None required.
Final Answer: Nice to meet you, Jihwan! How can I assist you today?

> Finished chain.


'Nice to meet you, Jihwan! How can I assist you today?'

In [32]:
agent_chain.run("Do you remember my name?")



> Entering new AgentExecutor chain...
Thought: The human just told me their name is Jihwan.
Final Answer: Yes, your name is Jihwan.

> Finished chain.


'Yes, your name is Jihwan.'